In [2]:
import pandas as pd


In [52]:
data =pd.read_csv('data.csv',nrows=100000)
ins = data.drop(columns=['id','Action'])
outs = data['Action']


In [53]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
# pipeline = Pipeline(
    
#     [
  
# TfidfVectorizer(),RandomForestClassifier()])
model = RandomForestClassifier()
# model = GaussianNB()
#model = DecisionTreeClassifier()#RandomForestClassifier()#GaussianNB()#LogisticRegression()

In [39]:
import numpy as np
class RandomForest:
    def __init__(self, n_trees=100, max_depth=None, min_samples_split=2):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        n_samples = X.shape[0]
        for i in range(self.n_trees):
            indices = np.random.choice(n_samples, n_samples, replace=True)
            X_subset = X[indices]
            y_subset = y[indices]
            tree = DecisionTreeClassifier(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            tree.fit(X_subset, y_subset)
            self.trees.append(tree)

    def predict(self, X):
        predictions = []
        for tree in self.trees:
            predictions.append(tree.predict(X))
        return np.round(np.mean(predictions, axis=0))
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        
    def fit(self, X, y):
        self.n_classes_ = len(np.unique(y))
        self.tree_ = self._grow_tree(X, y)
        
    def predict(self, X):
        return [self._predict(inputs) for inputs in X]
    
    def _predict(self, inputs):
        node = self.tree_
        while node.value is None:
            if inputs[node.feature_index] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.value
    
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        if (self.max_depth is not None and depth >= self.max_depth) or n_labels == 1:
            return Node(value=self._most_common_label(y))
        
        feature_indices = np.random.choice(n_features, n_features, replace=False)
        
        best_feature, best_threshold = self._best_criteria(X, y, feature_indices)
        
        left_indices = X[:, best_feature] <= best_threshold
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[~left_indices], y[~left_indices]
        
        left = self._grow_tree(X_left, y_left, depth + 1)
        right = self._grow_tree(X_right, y_right, depth + 1)
        
        return Node(best_feature, best_threshold, left, right)
    
    def _best_criteria(self, X, y, feature_indices):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feature_index in feature_indices:
            X_column = X[:, feature_index]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feature_index
                    split_thresh = threshold
        return split_idx, split_thresh
    
    def _information_gain(self, y, X_column, split_thresh):
        parent_entropy = self._entropy(y)
        left_indices = X_column <= split_thresh
        right_indices = X_column > split_thresh
        n = len(y)
        n_l, n_r = len(y[left_indices]), len(y[right_indices])
        e_l, e_r = self._entropy(y[left_indices]), self._entropy(y[right_indices])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r
        ig = parent_entropy - child_entropy
        return ig
    
    def _entropy(self, y):
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / counts.sum()
        entropy = sum(probabilities * -np.log2(probabilities))
        return entropy
    
    def _most_common_label(self, y):
        _, counts = np.unique(y, return_counts=True)
        return np.argmax(counts)


In [43]:
# from sklearn.preprocessing import LabelEncoder
# card1_n = LabelEncoder()
# card2_n = LabelEncoder()
# card3_n = LabelEncoder()
# card4_n = LabelEncoder()
# played_n = LabelEncoder()
# ins['card1']=card1_n.fit_transform(ins['Card 1'])
# ins['card2']=card1_n.fit_transform(ins['Card 2'])
# ins['card3']=card1_n.fit_transform(ins['Card 3'])
# ins['card4']=card1_n.fit_transform(ins['Card 4'])
# ins['playedn']=card1_n.fit_transform(ins['Played'])
# ins = ins.drop(columns=['Card 1','Card 2','Card 3','Card 4','Played'])

In [ ]:
def whotmodel():
  data =pd.read_csv('data.csv')
  ins = data.drop(columns=['id','Action'])
  outs = data['Action']
  arrs = [__ for _ in ins.values for __ in _]
  tokens  = {_:i for i,_ in enumerate(set(arrs)) }
  outtokens =  {_:i for i,_ in enumerate(set([_ for _ in list(outs.values)])) }
  insdata = []
  for i in ins.values:
      insdata.append([tokens[_] for _ in i])
  newdf = pd.DataFrame(data=insdata,columns=ins.columns)
  newdf.head()
  model = RandomForestClassifier()
  model.fit(newdf,outs)
  return model




In [71]:
import pandas as pd
from multiprocessing import Pool, cpu_count
from sklearn.ensemble import RandomForestClassifier

def whotmodel():
    data = pd.read_csv('data.csv')
    ins = data.drop(columns=['id','Action'])
    outs = data['Action']
    arrs = [__ for _ in ins.values for __ in _]
    tokens = {_: i for i, _ in enumerate(set(arrs))}
    outtokens = {_: i for i, _ in enumerate(set([_ for _ in list(outs.values)]))}
    insdata = []
    chunk_size = len(ins.values) // cpu_count()
    chunks = [ins.values[i:i+chunk_size] for i in range(0, len(ins.values), chunk_size)]
    def process_chunk(chunk):
        chunk_data = [[tokens[_] for _ in i] for i in chunk]
        return pd.DataFrame(data=chunk_data, columns=ins.columns)
    with Pool() as p:
        chunk_dfs = p.map(process_chunk, chunks)
    newdf = pd.concat(chunk_dfs)
    newdf.reset_index(drop=True, inplace=True)

    model = RandomForestClassifier()
    model.fit(newdf, outs)
    return model


2

In [72]:
mmodel = whotmodel()

AttributeError: ignored

In [54]:
arrs = [__ for _ in ins.values for __ in _]

tokens  = {_:i for i,_ in enumerate(set(arrs)) }
outtokens =  {_:i for i,_ in enumerate(set([_ for _ in list(outs.values)])) }

In [55]:
insdata = []
for i in ins.values:
    insdata.append([tokens[_] for _ in i])

In [56]:
newdf = pd.DataFrame(data=insdata,columns=ins.columns)
newdf.head()

,Card 1,Card 2,Card 3,Card 4,Played
0,10,35,39,32,10
1,10,35,39,32,35
2,10,35,39,32,39
3,10,35,39,32,32
4,10,35,39,32,13


In [57]:
len(newdf)

100000

In [58]:
model.fit(newdf,outs)


RandomForestClassifier()

In [49]:
def callmodel(cards,played,model1):
  cards.sort()
  card1,card2,card3,card4 = cards
  
 

  return model1.predict([[tokens[card1],tokens[card2],tokens[card3],tokens[card4],tokens[played]]])[0]

In [70]:
callmodel(['circle 1','circle 3','triangle 2','whot 20'],'sqaure 2',model)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'g o   m a r k e t'

In [47]:
model.predict([[0,0,0,0,1]])[0]

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


'circle 1'

0               circle 1
1               circle 1
2               circle 1
3               circle 1
4               circle 1
             ...        
995    g o   m a r k e t
996             circle 1
997             circle 2
998             circle 3
999    g o   m a r k e t
Name: Action, Length: 1000, dtype: object